In [ ]:
import PyDAQmx as dll
import numpy as np
import ctypes

DAQmx_Val_Cfg_Default             = ctypes.c_int32(-1)
DAQmx_Val_DoNotInvertPolarity     = ctypes.c_int32(0)
DAQmx_Val_GroupByChannel          = ctypes.c_int32(0)
DAQmx_Val_GroupByScanNumber       = ctypes.c_int32(1)
DAQmx_Val_ChanPerLine             = ctypes.c_int32(0)
DAQmx_Val_ChanForAllLines         = ctypes.c_int32(1)
DAQmx_Val_Acquired_Into_Buffer    = ctypes.c_int32(1)
DAQmx_Val_Ticks                   = ctypes.c_int32(10304)
DAQmx_Val_Rising                  = ctypes.c_int32(10280)
DAQmx_Val_Falling                 = ctypes.c_int32(10171)
DAQmx_Val_CountUp                 = ctypes.c_int32(10128)
DAQmx_Val_ContSamps               = ctypes.c_int32(10123)
DAQmx_Val_FiniteSamps             = ctypes.c_int32(10178)
DAQmx_Val_Hz                      = ctypes.c_int32(10373)
DAQmx_Val_Low                     = ctypes.c_int32(10214)
DAQmx_Val_Volts                   = ctypes.c_int32(10348)
DAQmx_Val_MostRecentSamp          = ctypes.c_uint32(10428)
DAQmx_Val_OverwriteUnreadSamps    = ctypes.c_uint32(10252)
DAQmx_Val_HWTimedSinglePoint      = ctypes.c_int32(12522)
DAQmx_Val_SampClk                 = ctypes.c_int32(10388)
DAQmx_Val_OnDemand                = ctypes.c_int32(10390)
DAQmx_Val_CurrReadPos             = ctypes.c_int32(10425)
DAQmx_Val_MostRecentSamp          = ctypes.c_int32(10428)
DAQmx_Val_OverwriteUnreadSamps    = ctypes.c_int32(10252)
DAQmx_Val_DoNotOverwriteUnreadSamps  = ctypes.c_int32(10159)

In [ ]:
CODevice_id = '/Dev2/Ctr0'
#CIDevice_id = '/Dev2/Ctr1'

COTask = dll.TaskHandle()

# Create taskHandle
dll.DAQmxCreateTask('', ctypes.byref(COTask))

period = 0.01   # seconds
f = 1/period
DutyCycle = 0.9

# Creates channel(s) to generate digital pulses that freq and dutyCycle define,
# and adds the channel to the task you specify with taskHandle.
dll.DAQmxCreateCOPulseChanFreq(
    COTask,                         # taskHandle
	CODevice_id,                    # Counter name
    '',                             # Channel name, physical channel name being used if left empty
    DAQmx_Val_Hz,                   # The amount of time in seconds to wait before generating the first pulse
    DAQmx_Val_Low,                  # The resting state of the output terminal.
    ctypes.c_double(0),             # initialDelay
    ctypes.c_double(f),             # The frequency at which to generate pulses.
	ctypes.c_double(DutyCycle)      # High time / (High time + Low time)
)

N = 10

# Sets only the number of samples to acquire or generate without specifying timing.
# Typically, you should use this function when the task does not require sample timing,
# such as tasks that use counters for buffered frequency measurement, buffered period
# measurement, or pulse train generation.
dll.DAQmxCfgImplicitTiming(
    COTask,                         # taskHandle
    DAQmx_Val_FiniteSamps,          # Specifies whether the task acquires or generates samples continuously,
                                    # or if it acquires or generates a finite number of samples.
    ctypes.c_ulonglong(N)           # The number of samples to acquire or generate for each channel in the task
                                    # if sampleMode is DAQmx_Val_FiniteSamps.
                                    # If sampleMode is DAQmx_Val_ContSamps, NI-DAQmx uses this value to determine
                                    # the buffer size.
)

0

In [ ]:
AOChannels = '/Dev2/ao0'
AOTask = dll.TaskHandle()
dll.DAQmxCreateTask('', ctypes.byref(AOTask))
v_min = 0
v_max = 5
PulseTrain_src = CODevice_id + 'InternalOutput'

# Creates channel(s) to generate voltage,
# and adds the channel(s) to the task you specify with taskHandle.
dll.DAQmxCreateAOVoltageChan(
    AOTask,                     # taskHandle
	AOChannels,                 # The names of the physical channels to use to create virtual channels.
                                # You can specify a list or range of physical channels.
    '',                         # The name(s) to assign to the created virtual channel(s)
	ctypes.c_double(v_min),     # The minimum value, in units
	ctypes.c_double(v_max),     # The maximum value, in units
	DAQmx_Val_Volts,            # The units in which to generate voltage
    ''                          # customScaleName
)

# Sets the source of the Sample Clock, the rate of the Sample Clock,
# and the number of samples to acquire or generate.
dll.DAQmxCfgSampClkTiming(
    AOTask,                     # taskHandle
	PulseTrain_src,				# The source terminal of the Sample Clock.
    							# To use the internal clock of the device, use NULL or use OnboardClock.
	ctypes.c_double(f),			# The sampling rate in samples per second per channel.
	DAQmx_Val_Falling,			# Specifies on which edge of the clock to acquire or generate samples.
    DAQmx_Val_FiniteSamps,		# Specifies whether the task acquires or generates samples continuously
    							# or if it acquires or generates a finite number of samples.
	ctypes.c_ulonglong(N)		# The number of samples to acquire or generate for each channel in the
    							# task if sampleMode is DAQmx_Val_FiniteSamps. 
)

autoStart 	= False
RWTimeout 	= 1.0
data		= np.linspace(v_min, v_max, N)
AONwritten  = ctypes.c_int32()

dll.DAQmxWriteAnalogF64(
    AOTask,
	N,
	autoStart,
	RWTimeout,
	DAQmx_Val_GroupByChannel,	# Specifies how the samples are arranged
	data,
	dll.byref(AONwritten), 
	None
)
print(AONwritten)

0

In [7]:
dll.DAQmxStartTask(COTask)

0

In [8]:
dll.DAQmxStopTask(COTask)

0